# Funds and SDG/SDT Sankeys



## Libs and setup


In [ ]:
!pip install --upgrade plotly
!pip install kaleido

In [ ]:
import requests
import plotly.graph_objects as go
from collections import defaultdict
import textwrap



SDG_GOAL_MAP = {
    '1': {
        'name': '1. No poverty',
        'node_colour': '#e5243b',
        'link_colour': 'rgba(229, 36, 59, 0.5)',
    },
    '2': {
        'name': '2. Zero hunger',
        'node_colour': '#DDA63A',
        'link_colour': 'rgba(221, 166, 58, 0.5)',
    },
    '3': {
        'name': '3. Good health and wellbeing',
        'node_colour': '#4C9F38',
        'link_colour': 'rgba(76, 159, 56, 0.5)',
    },
    '4': {
        'name': '4. Quality education',
        'node_colour': '#C5192D',
        'link_colour': 'rgba(197, 25, 45, 0.5)',
    },
    '5': {
        'name': '5. Gender equality',
        'node_colour': '#FF3A21',
        'link_colour': 'rgba(255, 58, 33, 0.5)',
    },
    '6': {
        'name': '6. Clean water and sanitation',
        'node_colour': '#26BDE2',
        'link_colour': 'rgba(38, 189, 226, 0.5)',
    },
    '7': {
        'name': '7. Affordable and clean energy',
        'node_colour': '#FCC30B',
        'link_colour': 'rgba(252, 195, 11, 0.5)',
    },
    '8': {
        'name': '8. Work and economic growth',
        'node_colour': '#A21942',
        'link_colour': 'rgba(162, 25, 66, 0.5)',
    },
    '9': {
        'name': '9. Industry, innovation and infrastructure',
        'node_colour': '#FD6925',
        'link_colour': 'rgba(253, 105, 37, 0.5)',
    },
    '10': {
        'name': '10. Reduced inequalities',
        'node_colour': '#DD1367',
        'link_colour': 'rgba(221, 19, 103, 0.5)',
    },
    '11': {
        'name': '11. Sustainable cities and communities',
        'node_colour': '#FD9D24',
        'link_colour': 'rgba(253, 157, 36, 0.5)',
    },
    '12': {
        'name': '12. Responsible consumption and production',
        'node_colour': '#BF8B2E',
        'link_colour': 'rgba(191, 139, 46, 0.5)',
    },
    '13': {
        'name': '13. Climate action',
        'node_colour': '#3F7E44',
        'link_colour': 'rgba(63, 126, 68, 0.5)',
    },
    '14': {
        'name': '14. Life below water',
        'node_colour': '#0A97D9',
        'link_colour': 'rgba(10, 151, 217, 0.5)',
    },
    '15': {
        'name': '15. Life on land',
        'node_colour': '#56C02B',
        'link_colour': 'rgba(86, 192, 43, 0.5)',
    },
    '16': {
        'name': '16. Peace, justice and strong institutions',
        'node_colour': '#00689D',
        'link_colour': 'rgba(0, 104, 157, 0.5)',
    },
    '17': {
        'name': '17. Partnerships for the goals',
        'node_colour': '#19486A',
        'link_colour': 'rgba(25, 72, 106, 0.5)',
    },
}


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id=None):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        if public_id:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        else:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False


def _get_sdg_values_for_fund_data(fund_data):
    primary_value = fund_data.get('purpose_and_classifications',{}).get('primary_sdg_goal',{}).get('value','')
    secondary_values = fund_data.get('purpose_and_classifications',{}).get('secondary_sdg_goals',{}).get('value','')
    sdg_values = set()
    if primary_value and isinstance(primary_value, int):
        sdg_values.add(str(primary_value))
    if primary_value and isinstance(primary_value, str) and primary_value.strip():
        sdg_values.add(primary_value.strip())
    if secondary_values:
        [sdg_values.add(i.strip()) for i in str(secondary_values).split(',') if i.strip()]
    return list(sdg_values)
        
def sankey_viz(funds, sankey_height=400, filename='out.png', textwrap_width=50):
    # Pass 1: Make sure we have all the nodes listed
    funds_id_to_name = {}
    sdg_nodes = set()
    for fund_id,fund_data in funds.items():
        for sdg_value in _get_sdg_values_for_fund_data(fund_data):
            funds_id_to_name[fund_id] = fund_data.get('name',{}).get('value','')
            sdg_nodes.add(sdg_value)
    
    # Pass 2: Get links
    links = defaultdict(dict)
    for fund_id,fund_data in funds.items():
        for sdg_value in _get_sdg_values_for_fund_data(fund_data):
            links[fund_id][sdg_value] = 1

    # Turn into data suitable for sankey
    node_labels = []
    node_colours = []
    fund_nodes_idx = {}
    sdg_nodes_idx = {}
    for fund_id, fund_name in funds_id_to_name.items():
        node_labels.append(textwrap.shorten(fund_id + ": "+ fund_name, textwrap_width))
        node_colours.append("black")
        fund_nodes_idx[fund_id] = len(node_labels) - 1
    for sdg_node in sdg_nodes:
        node_labels.append(textwrap.shorten(SDG_GOAL_MAP[sdg_node]['name'], textwrap_width))
        node_colours.append(SDG_GOAL_MAP[sdg_node]['node_colour'])
        sdg_nodes_idx[sdg_node] = len(node_labels) - 1

    links_for_sankey = []
    for fund_id in links.keys():
        for sdg in links[fund_id].keys():
            links_for_sankey.append([fund_id, sdg, links[fund_id][sdg]])

    sankey_node_data = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = node_labels,
      color = node_colours
    )

    sankey_link_data = dict(
      source = [fund_nodes_idx[l[0]] for l in links_for_sankey],
      target = [sdg_nodes_idx[l[1]] for l in links_for_sankey],
      value = [l[2] for l in links_for_sankey],
      color = [SDG_GOAL_MAP[l[1]]['link_colour'] for l in links_for_sankey]
    )

    # make Sankey
    fig = go.Figure(
        data=go.Sankey(
            node=sankey_node_data,
            link=sankey_link_data,
            arrangement='perpendicular',
        ),
    )

    fig.update_layout(
        height=sankey_height,
    )

    fig.show()
    fig.write_image(filename)




## Get Fund data
Call the INDIGO API 'fund' endpoint and retrieve the data used for the plot.

By default this will get data for all the funds, but you can pass a list of fund ID's and get only some. See the comments in the code.

In [ ]:
# Call the API and pull down the data for each fund
# and store in a fund for use later.
# 
# You can set public_ids to some funds only
# eg:
# public_ids = ['INDIGO-FUND-0001', ]
# or pass an empty list, in which case it will get data from all funds
# eg:
# public_ids = []

public_ids = []
endpoint = 'fund'

if not public_ids:
    for fund_data in api_get_item(endpoint).get('funds'):
        if fund_data.get('public'):
            public_ids.append(fund_data.get('id'))

funds_data = {}

for fund_id in public_ids:
    print("Getting Fund " + fund_id)
    funds_data[fund_id] = api_get_item(endpoint, fund_id).get('fund',{}).get('data',{})

## Basic Sankey

In [ ]:
sankey_viz(funds_data, sankey_height=2000, filename='sankey.png', textwrap_width=45)
